In [ ]:
%load_ext autoreload
%autoreload 2
from arpes.all import *
import os.path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import numpy as np
%matplotlib inline

In [ ]:
# Define directory
path = os.path.abspath(r'C:\Users\hellbrue\Documents\Bi2223_1st_beamtime')

# Different parameters used to set up scans at SLS
scan_type = '*'
direction = 'X'
phi = '*'
energy = '*'
polarization = '*'
temp = '*'
e_pass = '*'+'eVpass'
comment = '*'
run = '*'.zfill(4)

# Put together string for finding file
parameters = [scan_type, direction, phi, energy, polarization, temp, e_pass, comment, run]
if comment == '':
    parameters.remove(comment)
file_type = 'h5'
file_name = '_'.join(parameters)
file = '.'.join([file_name , file_type])

au_full_file = os.path.join(path, file)
file

In [ ]:
import glob
# Load all files with above specified parameters, * is a placeholder and imports all files in its place
au_files = glob.glob(au_full_file)
print(len(au_files))
au_files

In [20]:
au_raw_data_list = {}
for file in au_files:
    #data_list.append(load_data(file, location="ULTRA"))
    data = load_data(file, location="ULTRA")
    # k_data = convert_to_kspace(data.spectrum)
    file_name = file.replace(path, '')[1:-3]
    au_raw_data_list.update({file_name : data})

poly Au used to determine Fermi level. Determine Fermiedge and apply to data below

In [ ]:
au_lt = au_raw_data_list['EDC_X_P95_25eV_UNKN_LT_5eVpass_GoldRef11K_0007']
au_lt.S.plot()

In [ ]:
from arpes.analysis.general import rebin

binned = rebin(au_lt, phi=10).spectrum
binned.plot()

In [ ]:
from arpes.all import AffineBroadenedFD, broadcast_model
fermi_level = binned.sel(phi=slice(-0.2,0.2), eV=slice(-0.2,0.1))
results = broadcast_model(AffineBroadenedFD, fermi_level, "phi")

fig, ax = plt.subplots()
fermi_level.plot(ax=ax)
ax.scatter(*results.F.p("fd_center").G.to_arrays(), color="red")
ax.axhline(y=0, color='k', linestyle='--', linewidth=1)

In [ ]:
from arpes.fits.fit_models import LinearModel
lin_mod = LinearModel().guess_fit(results.F.p("fd_center"))
lin_mod.plot()
plt.show()

In [ ]:
edc = au_lt.spectrum
edge = lin_mod.eval(x=edc.phi)

corrected_edc = edc.G.shift_by(edge, "eV")

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
corrected_edc.plot(ax=axes[0], vmax=10)
edc.plot(ax=axes[1], vmax=20)
# plot vertical lines for y = 0
for ax in axes:
    ax.set_title("")
    ax.axhline(y=0, color='k', linestyle='--', linewidth=1)

In [ ]:
# Define directory
path = os.path.abspath(r'C:\Users\hellbrue\Documents\Bi2223_1st_beamtime')

# Different parameters used to set up scans at SLS
scan_type = 'FS'
direction = 'N'
phi = 'P50'
energy = '*'
polarization = 'CIRCM'
temp = '*'
e_pass = '*'+'eVpass'
comment = 'zoom'
run = '0'.zfill(4)

# Put together string for finding file
parameters = [scan_type, direction, phi, energy, polarization, temp, e_pass, comment, run]
if comment == '':
    parameters.remove(comment)
file_type = 'h5'
file_name = '_'.join(parameters)
file = '.'.join([file_name , file_type])

bi2223_full_file = os.path.join(path, file)
file

In [ ]:
import glob
# Load all files with above specified parameters, * is a placeholder and imports all files in its place
bi2223_files = glob.glob(bi2223_full_file)
print(len(bi2223_files))
bi2223_files

In [32]:
bi2223_data_list = {}
for file in bi2223_files:
    #data_list.append(load_data(file, location="ULTRA"))
    raw_data = load_data(file, location="ULTRA")
    edge = lin_mod.eval(x=raw_data.spectrum.phi)
    corrected_data = raw_data.spectrum.G.shift_by(edge, "eV")
    corrected_data = rebin(corrected_data, phi=3, beta=3)
    corrected_data.S.apply_offsets({"beta": -0.05, "phi": 0.15})
    k_data = convert_to_kspace(corrected_data)
    file_name = file.replace(path, '')[1:-3]
    bi2223_data_list.update({file_name : k_data})

Initalize datasets

In [33]:
lt_fs_zoom = bi2223_data_list['FS_N_P50_26p5eV_CIRCM_LT_5eVpass_zoom_0000']


Overview of the LT P5 Dataset

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
lt_fs_zoom.sel(eV=slice(-0.02,0.02), kx=slice(-0.6,-0.09), ky=slice(-0.23,0.25)).sum("eV").plot(ax=ax, cmap='PuBu_r', vmin=3, vmax=50, rasterized=True)

ax.set_title('')
ax.set_ylabel('k$_y$ ($\AA^{-1}$)', fontsize=18)
ax.set_xlabel('k$_x$ ($\AA^{-1}$)', fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=16)

ax.collections[0].colorbar.remove()
cbar = plt.colorbar(ax.collections[0], ax=ax, fraction=0.1, pad=0.05, aspect=25)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.ax.text(0.6, 1.0, 'Max', ha='center', va='bottom', transform=cbar.ax.transAxes, fontsize=12)
cbar.ax.text(0.6, -0.06, 'Min', ha='center', va='bottom', transform=cbar.ax.transAxes, fontsize=12)
cbar.set_ticks([])
# set x and y axis to be equal
# plt.savefig('FS_hybrid_zoom.pdf', bbox_inches='tight')
plt.show()